# 크롤링 실습

## 크롤링 실습을 위한 준비 (주로 로컬 환경에서 진행함)


In [1]:
!pip3 install requests
!pip3 install bs4
!pip3 install selenium

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1264 sha256=81da1a83cf28539a0ce58b88bb5b0ab9283df801b6a35de305f8965d2ef51864
  Stored in directory: c:\users\wuhen\appdata\local\pip\cache\wheels\e4\62\1d\d4d1bc4f33350ff84227f89b258edb552d604138e3739f5c83
Successfully built bs4


## GET 요청하기

In [2]:
import requests

url = 'https://search.naver.com/search.naver?ie=UTF-8&query=%EC%A0%9C%EC%A3%BC%EB%8F%84&sm=chr_hty'
res = requests.get(url)

print(res) # 응답 객체
res.text # HTML 텍스트

<Response [200]>


'<!doctype html> <html lang="ko"><head> <meta charset="utf-8"> <meta name="referrer" content="always">  <meta name="format-detection" content="telephone=no,address=no,email=no"> <meta name="viewport" content="width=device-width,initial-scale=1.0,maximum-scale=2.0"> <meta property="og:title" content="제주도 : 네이버 통합검색"/> <meta property="og:image" content="https://ssl.pstatic.net/sstatic/search/common/og_v3.png"> <meta property="og:description" content="\'제주도\'의 네이버 통합검색 결과입니다."> <meta name="description" lang="ko" content="\'제주도\'의 네이버 통합검색 결과입니다."> <title>제주도 : 네이버 통합검색</title> <link rel="shortcut icon" href="https://ssl.pstatic.net/sstatic/search/favicon/favicon_191118_pc.ico">  <link rel="search" type="application/opensearchdescription+xml" href="https://ssl.pstatic.net/sstatic/search/opensearch-description.https.xml" title="Naver" /><script> if (top.frames.length!=0 || window!=top) window.open(location, "_top"); </script><link rel="stylesheet" type="text/css" href="https://ssl.pstatic.n

#### HTTP 상태 코드
 - 1xx (정보): 요청을 받았으며 프로세스를 계속한다
 - 2xx (성공): 요청을 성공적으로 받았으며 인식했고 수용하였다
 - 3xx (리다이렉션): 요청 완료를 위해 추가 작업 조치가 필요하다
 - 4xx (클라이언트 오류): 요청의 문법이 잘못되었거나 요청을 처리할 수 없다
 - 5xx (서버 오류): 서버가 명백히 유효한 요청에 대해 충족을 실패했다

[출처: 위키피디아](https://ko.wikipedia.org/wiki/HTTP_%EC%83%81%ED%83%9C_%EC%BD%94%EB%93%9C)

In [3]:
res.status_code # HTTP 상태 코드

200

## BeautifulSoup을 이용한 크롤링

In [4]:
# 임시 html 텍스트
html = '''
<html>
  <head>
    <title>Example HTML</title>
  </head>
  <body>
    <div id='first' class='Example'>
      <h3 title='Good Content Title'>Contents Title</h3>
      <p>Test contents</p>
    </div>
    <div id='second' class='Example'>
      <p>Text1</p>
      <p>Text2</p>
      <p>Text3</p>
    </div>
  </body>
</html>
'''

In [5]:
from bs4 import BeautifulSoup

# 첫 번째 인자: 파싱할 객체
# 두 번째 인자: 사용할 parser(구문 분석기) (html.parser, lxml, html5lib 등이 있다)
soup = BeautifulSoup(html, 'html.parser')

In [6]:
soup


<html>
<head>
<title>Example HTML</title>
</head>
<body>
<div class="Example" id="first">
<h3 title="Good Content Title">Contents Title</h3>
<p>Test contents</p>
</div>
<div class="Example" id="second">
<p>Text1</p>
<p>Text2</p>
<p>Text3</p>
</div>
</body>
</html>

find(), find_all()

태그 검색   
- find(tag, attributes, recursive, text, keywords)   
- find_all(tag, attributes, recursive, text, limit, keywords)   

get_text()

태그 내 텍스트 반환(부모 태그의 경우 자식 태그의 텍스트를 모두 반환)

In [ ]:
tag = soup.find('h3') # h3 태그만 가져오되, 여러 개일 경우가장 윗줄 (1개)
print(tag)
tag.get_text()

<h3 title="Good Content Title">Contents Title</h3>


'Contents Title'

In [11]:
# 부모 태그의 경우 자식 태그의 텍스트를 모두 반환
tag_parent = soup.find('div')
print(tag_parent)
print()
tag_parent.get_text()
# \n도 같이 들어간다는걸 볼 수 있다

<div class="Example" id="first">
<h3 title="Good Content Title">Contents Title</h3>
<p>Test contents</p>
</div>



'\nContents Title\nTest contents\n'

In [13]:
tag = soup.find('p') # 여러 개일 경우 가장 위줄(1개)
print(tag)
tag.get_text()

<p>Test contents</p>


'Test contents'

In [15]:
# find_all은 list 형태로 값 반환
soup.find_all('p')

[<p>Test contents</p>, <p>Text1</p>, <p>Text2</p>, <p>Text3</p>]

In [16]:
soup.find_all('p')[2].get_text()

'Text2'

In [17]:
# 개수 제한
soup.find_all('p', limit=2) # limite parameter는 개수 제한, 여기에서는 2니깐 2개

[<p>Test contents</p>, <p>Text1</p>]

In [18]:
# 여러 태그
soup.find_all({'p', 'h3'})

[<h3 title="Good Content Title">Contents Title</h3>,
 <p>Test contents</p>,
 <p>Text1</p>,
 <p>Text2</p>,
 <p>Text3</p>]

In [19]:
soup.find('div')

<div class="Example" id="first">
<h3 title="Good Content Title">Contents Title</h3>
<p>Test contents</p>
</div>

In [20]:
soup.find('div').get_text().replace('\n', '') # get_text()는 string 반환

'Contents TitleTest contents'

In [21]:
# 특정 속성을 가진 태그 검색
soup.find('div', {'id': 'second'}) #id가 second인 div를 가져와라

<div class="Example" id="second">
<p>Text1</p>
<p>Text2</p>
<p>Text3</p>
</div>

In [22]:
# keyword 매개변수 이용
soup.find('div', id='second')

<div class="Example" id="second">
<p>Text1</p>
<p>Text2</p>
<p>Text3</p>
</div>

In [23]:
soup.find('div', id='second', class_='Example') # id가 second이면서 동시에 calss는 Example

<div class="Example" id="second">
<p>Text1</p>
<p>Text2</p>
<p>Text3</p>
</div>

In [24]:
# 속성이 여러 개일 경우
attrs = {
    'id': 'second',
    'class': 'Example'
}
soup.find('div', attrs)

<div class="Example" id="second">
<p>Text1</p>
<p>Text2</p>
<p>Text3</p>
</div>

네이버 금융 주요뉴스 헤드라인 크롤링 예시

In [29]:
url = "https://finance.naver.com/"
res = requests.get(url)

# 첫번째 인자를 텍스트로 넣어줘야 함
soup = BeautifulSoup(res.text, 'html.parser') # 문서 구조에 따라 'html.parser', 'lxml', 'xml' 등 다양한 구조 분석 방식을 사용할 수 있음

# 주요뉴스 섹션 검색
strategy = soup.find('div', class_="section_strategy")

# 주요뉴스 헤드라인을 저장할 리스트
strategies = []

# 주요뉴스에 있는 헤드라인 텍스트 정보 추출한 뒤 리스트에 저장
for i in range(len(strategy.find_all('a'))-1):
  strategies.append(strategy.find_all('a')[i].get_text())

# 결과 확인
for headline in strategies:
  print(headline)


美 인플레 둔화 기대…환율, 1293.7원으로 2주래 최저[외환마감]
코스피, 6거래일 만에 반등…2560선 장 종료[마감시황]
코스피, 돌아온 외인·기관에 2% 상승…삼성전자 '7만전자' 안착[시황종합]
[코스닥 마감] 돌아온 외인에 2% 상승…엔터·게임주 고진감래
뉴욕증시 훈풍에…닛케이 0.28%↑[Asia오전]
[굿모닝 증시]美 반도체 강세…코스피 상승 출발 전망


## Selenium을 이용한 크롤링 - 이 부분은 로컬 환경에서 코드를 실행해 봅시다.

- 크롤링은 Colab이 아닌 개인 로컬환경에서 진행하는 것이 대다수
- Colab에서 Selenium을 진행할 경우 브라우저 창을 띄울 수 없다 -> 크롤링이 잘 진행되고 있는지 직관적으로 확인하기 어려움
- Colab에서 Selenium 패키지를 설치하여 실행하는 코드를 제대로 작성해도 작동하지 않는 경우 발생 -> 따라서 Selenium을 이용하는 경우 Chrome WebDriver를 본인 PC에 맞게 설치하여 로컬환경에서 크롤링을 진행하도록 합시다!   

In [31]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

'apt-get'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'apt'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'cp'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [37]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys # 키보드 키 이벤트를 사용하기 위한 모듈
from selenium.webdriver.common.by import By # 웹 요소를 검색하는 방법을 지정하는 방식을 위한 모듈입니다. By.ID, By.NAME, By.CLASS_NAME 등이 있다

options = webdriver.ChromeOptions() # 크롬 드라이버의 옵션을 설정하는 객체를 생성한다

#내부 창을 띄울 수 없으므로 설정 ( GUI 없이 작동하는 'headless' 모드를 활성화하는 옵션, 이 옵션을 설정하면 웹 브라우저가 실제로 화면에 나타나지 않고 백그라운드에서 동작하게 된다)
options.add_argument('--headless') 

# Chrome이 Sandboxing 모드를 비활성화하게 하는 옵션, Sandboxing 모드는 프로세스를 격리하여 보안을 강화하는 기능이지만, 환경에 따라서는 문제를 일으킬 수 있어 비활성화시키는 경우가 있다
options.add_argument('--no-sandbox')

# /dev/shm'을 사용하지 않도록 하는 옵션. '/dev/shm'은 공유 메모리를 위한 임시 파일 시스템이지만, 일부 환경에서는 이 장소가 부족하여 크롬이 종료되는 문제가 있을 수 있다. 이 옵션은 그러한 문제를 피하기 위해 사용된다.
options.add_argument('--disable-dev-shm-usage')

# 앞서 설정한 옵션을 적용하여 크롬 웹 드라이버 객체를 생성한다. 'chromedriver'는 크롬 웹 드라이버의 위치를 나타내며, 상대 경로 혹은 절대 경로가 될 수 있다. 이 드라이버 객체를 이용해 웹 브라우저를 제어할 수 있게 된다
driver = webdriver.Chrome('chromedriver',options=options)

In [36]:
# webdriver에 옵션 적용 가능
options = webdriver.ChromeOptions()

# 브라우저 창이 안뜨고 실행
options.add_argument('headless')
# 브라우저 사이즈 설정
options.add_argument('window-size=1920x1080')

# 옵션 적용
driver = webdriver.Chrome(path, options=options)

NameError: name 'path' is not defined

In [40]:
driver.get('https://www.naver.com/')

# id 속성으로 검색창 찾기
search = driver.find_element(By.ID , 'query')

# 검색어 입력
search.send_keys('제주도')
# Keys.RETURN = Enter 입력
search.send_keys(Keys.RETURN)


# a 태그의 텍스트로 찾기
search = driver.find_element(By.LINK_TEXT,'이미지') # 태그 안의 엘리먼트의 내용이 '이미지'인 엘리먼트를 찾는다.
# 클릭
search.click()

\* 셀레니움 사용 시 브라우저에서 해당 웹 페이지의 요소들을 로드하는 데 시간이 좀 걸리기 때문에, element가 존재하지 않는다는 error가 발생할 수 있다.   
그럴 경우 해당 요소가 전부 준비가 될 때까지 대기해야 한다.

```python
driver.implicitly_wait(time_to_wait=5)
```
5초가 지나도 없으면 에러

-> 찾으려는 element가 로드될 때까지 지정한 시간만큼 대기할 수 있도록 설정   
위 방법은 암시적 대기 방식으로, webdriver에 영구적으로 작용된다.   
time_to_wait=5: 5초까지 기다림

https://workingwithpython.com/selenium-waits/

\* cf. 브라우저 로드가 완성될 때까지 단순히 n초를 기다리는 방식은 time.sleep(n) 코드를 통해 실행가능, 각 PC 환경에 따라 브라우저 로드 속도의 차이가 각각 다르므로 driver.implicitly_wait(5)를 사용하는게 속도 측면에서 더 효율적일 수 있다.

```python
time.sleep(5)
```

In [44]:
# 현재 열려있는 페이지 url 찾기
url = driver.current_url
url


InvalidSessionIdException: Message: invalid session id
Stacktrace:
Backtrace:
	GetHandleVerifier [0x009EA813+48355]
	(No symbol) [0x0097C4B1]
	(No symbol) [0x00885220]
	(No symbol) [0x008A7C7C]
	(No symbol) [0x008CA636]
	(No symbol) [0x008C70CD]
	(No symbol) [0x008C6B3A]
	(No symbol) [0x00865311]
	(No symbol) [0x0086571E]
	(No symbol) [0x00865AFA]
	GetHandleVerifier [0x00C4AABD+2539405]
	GetHandleVerifier [0x00C8A78F+2800735]
	GetHandleVerifier [0x00C8456C+2775612]
	GetHandleVerifier [0x00A751E0+616112]
	(No symbol) [0x00985F8C]
	(No symbol) [0x00864F64]
	(No symbol) [0x00864912]
	GetHandleVerifier [0x00CB159C+2959980]
	BaseThreadInitThunk [0x772B00C9+25]
	RtlGetAppContainerNamedObjectPath [0x77E27B4E+286]
	RtlGetAppContainerNamedObjectPath [0x77E27B1E+238]


In [43]:
# 브라우저 닫기
driver.close()

# Selenium을 활용한 크롤링 예시:네이버 토요일, 일요일 웹툰 정보 크롤링
- 로컬 환경에서 진행

### 과제 url

url => https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1

과제: 무신사 코디맵 페이지(많은 댓글 수 기준 정렬)에 나와있는 60개의 코디맵에 대하여 각각 ['codimap_category', 'codimap_title', 'codimap_date', 'views', 'comment_numbers'] 정보를 가져와서 csv 형태로 만든 다음, 해당 코드(.ipynb)와 csv 파일을 본인 github 과제 repository에 제출

Challenge:
1. 각 코디맵마다 클릭한 뒤 다시 이전페이지로 돌아오는 코드를 반복문에 추가하여 ['codimap_explain', 'codimap_hashtag', 'codimap_imgurl'] 정보도 가져오기
2.  page url을 format 함수를 이용해 수정한 뒤 반복문을 통해 1페이지부터 5페이지까지의 코디맵 정보 가져오기

# Challenge는 Optional Task로 필수적으로 해야하는 것은 아닙니다. 좀 더 심화된 크롤링을 원할경우 시도해보면 좋을 것 같아요!